In [1]:
import numpy as np
import matplotlib.pyplot as plt
from hyperspy.api import load
from pixstem.api import PixelatedSTEM
%matplotlib qt

from scipy.signal.signaltools import wiener
from skimage.feature import match_template

file = load("input6.blo")


In [2]:
s = PixelatedSTEM(file.inav[30, 30])
image = np.array(s)

In [3]:
from skimage.restoration import denoise_nl_means, estimate_sigma

sigma_est = np.mean(estimate_sigma(image, ))
nlm = denoise_nl_means(image, h=1.15*sigma_est, fast_mode=True, patch_size=1, patch_distance=6, )

# fig = plt.figure(figsize=(8, 3))
# ax1 = plt.subplot(1, 2, 1)
# ax2 = plt.subplot(1, 2, 2)
# ax1.imshow(image)
# ax2.imshow(nlm)

In [4]:
newImg = wiener(nlm, 5, 3)
t = PixelatedSTEM(newImg)
##############

template2 = nlm[260:315, 255:310]
result2 = match_template(nlm, template2, pad_input=True)
kl = np.unravel_index(np.argmax(result2), result2.shape)
m, n = kl[::-1]

template1 = image[260:315, 255:310]
result1 = match_template(image, template1, pad_input=True)
cd = np.unravel_index(np.argmax(result1), result1.shape)
e, f = cd[::-1]

template = newImg[260:315, 255:310]
result = match_template(newImg, template, pad_input=True)
ij = np.unravel_index(np.argmax(result), result.shape)
x, y = ij[::-1]

###############
list = []
for i in range(len(result)):
    for j in range(len(result[i])):
        if (result[i][j] > 0.87):#change correlation value
            list.append((i, j))
list1 = []
for i in range(len(result1)):
    for j in range(len(result1[i])):
        if (result1[i][j] > 0.87):#change correlation value
            list1.append((i, j))
list2 = []
for i in range(len(result2)):
    for j in range(len(result2[i])):
        if (result2[i][j] > 0.87):#change correlation value
            list2.append((i, j))


def distance(x1, y1, x2, y2):
    return np.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))

##############

i = 0
l = []
while i < len(list):
    j = 0
    temp = []
    point = list[i]
    while j < len(list):
        if distance(point[0], point[1], list[j][0], list[j][1]) < 15:
            temp.append(list[j])
            list.pop(j)
        else:
            j = j + 1
    max = 0
    pnt = temp[0]
    for j in range(len(temp)):
        if (result[pnt[0]][pnt[1]] < result[temp[j][0]][temp[j][1]]):
            max = result[temp[j][0]][temp[j][1]]
            pnt = temp[j]
    l.append(pnt)

i = 0
l1 = []
while i < len(list1):
    j = 0
    temp = []
    point = list1[i]
    while j < len(list1):
        if distance(point[0], point[1], list1[j][0], list1[j][1]) < 15:
            temp.append(list1[j])
            list1.pop(j)
        else:
            j = j + 1
    max = 0
    pnt = temp[0]
    for j in range(len(temp)):
        if (result1[pnt[0]][pnt[1]] < result1[temp[j][0]][temp[j][1]]):
            max = result1[temp[j][0]][temp[j][1]]
            pnt = temp[j]
    l1.append(pnt)

i = 0
l2 = []
while i < len(list2):
    j = 0
    temp = []
    point = list2[i]
    while j < len(list2):
        if distance(point[0], point[1], list2[j][0], list2[j][1]) < 15:
            temp.append(list2[j])
            list2.pop(j)
        else:
            j = j + 1
    max = 0
    pnt = temp[0]
    for j in range(len(temp)):
        if (result2[pnt[0]][pnt[1]] < result2[temp[j][0]][temp[j][1]]):
            max = result2[temp[j][0]][temp[j][1]]
            pnt = temp[j]
    l2.append(pnt)

###############
fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2)
ax3 = plt.subplot(1, 3, 3)
# ax4 = plt.subplot(2, 2, 4)

ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_axis_off()
ax1.set_title('Original')

ax2.imshow(nlm, cmap=plt.cm.gray)
ax2.set_axis_off()
ax2.set_title('Non-local Means')

ax3.imshow(newImg, cmap=plt.cm.gray)
ax3.set_axis_off()
ax3.set_title('Wiener after Non-local Means')

# ax3.imshow(template1, cmap=plt.cm.gray)
# ax3.set_axis_off()
# ax3.set_title('original template')

# ax4.imshow(image, cmap=plt.cm.gray)
# ax4.set_axis_off()
# ax4.set_title('original image')

for i in l:
    a, b = i
    circ = plt.Circle((b, a), 1, edgecolor='r', facecolor='none')
    ax3.add_patch(circ)
for i in l1:
    a, b = i
    circ = plt.Circle((b, a), 1, edgecolor='r', facecolor='none')
    ax1.add_patch(circ)
for i in l2:
    a, b = i
    circ = plt.Circle((b, a), 1, edgecolor='r', facecolor='none')
    ax2.add_patch(circ)

plt.show()